In [ ]:
import gzip
import json
import dateutil.parser
import random
import numpy as np
from collections import defaultdict

In [ ]:
import homework2

In [ ]:
root = "."

In [ ]:
def parseData(fname):
    for l in open(fname):
        yield eval(l)

In [ ]:
data = list(parseData(root + "/beer_50000.json"))

In [ ]:
random.seed(0)
random.shuffle(data)

In [ ]:
dataTrain = data[:25000]
dataValid = data[25000:37500]
dataTest = data[37500:]

In [ ]:
categoryCounts = defaultdict(int)
for d in data:
    categoryCounts[d['beer/style']] += 1

In [ ]:
categories = [c for c in categoryCounts if categoryCounts[c] > 1000]

In [ ]:
catID = dict(zip(list(categories),range(len(categories))))

In [ ]:
catID

In [ ]:
def testQ1():
    mod, validBER, testBER = homework2.Q1(catID, dataTrain, dataValid, dataTest)
    return validBER, testBER

In [ ]:
testQ1()

In [ ]:
def testQ2():
    mod, validBER, testBER = homework2.Q2(catID, dataTrain, dataValid, dataTest)
    return validBER, testBER

In [ ]:
#testQ2()

In [ ]:
def testQ3():
    mod, validBER, testBER = homework2.Q3(catID, dataTrain, dataValid, dataTest)
    return validBER, testBER

In [ ]:
#testQ3()

In [ ]:
def testQ4():
    testBER_noCat, testBER_noReview, testBER_noLength = homework2.Q4(1, catID, dataTrain, dataValid, dataTest)
    return testBER_noCat, testBER_noReview, testBER_noLength

In [ ]:
#testQ4()

In [ ]:
path = root + "/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
f = gzip.open(path, 'rt', encoding="utf8")

header = f.readline()
header = header.strip().split('\t')

In [ ]:
header

In [ ]:
review_dataset = []

pairsSeen = set()

for line in f:
    fields = line.strip().split('\t')
    d = dict(zip(header, fields))
    ui = (d['customer_id'], d['product_id'])
    if ui in pairsSeen:
        print("Skipping duplicate user/item:", ui)
        continue
    pairsSeen.add(ui)
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    review_dataset.append(d)

In [ ]:
reviewDataTrain = review_dataset[:int(len(review_dataset)*0.9)]
reviewDataTest = review_dataset[int(len(review_dataset)*0.9):]

In [ ]:
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated
itemNames = {}
ratingDict = {} # To retrieve a rating for a specific user/item pair
reviewsPerUser = defaultdict(list)

for d in reviewDataTrain:
    user,item = d['customer_id'], d['product_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    reviewsPerUser[user].append(d)

for d in review_dataset:
    user,item = d['customer_id'], d['product_id']
    ratingDict[(user,item)] = d['star_rating']
    itemNames[item] = d['product_title']

In [ ]:
def testQ5():
    # Note: your solution is autograded based on the relevance scores, not the items themselves
    return homework2.mostSimilar("B00KCHRKD6", 10, usersPerItem)

In [ ]:
#testQ5()

In [ ]:
ratingMean = homework2.getMeanRating(reviewDataTrain)

userAverages = homework2.getUserAverages(itemsPerUser, ratingDict)

itemAverages = homework2.getItemAverages(usersPerItem, ratingDict)

In [ ]:
def testQ6():
    alwaysPredictMean = [ratingMean for d in reviewDataTest]
    
    simPredictions = [homework2.predictRating(d['customer_id'],
                                              d['product_id'],
                                              ratingMean,
                                              reviewsPerUser,
                                              usersPerItem,
                                              itemsPerUser,
                                              userAverages,
                                              itemAverages) for d in reviewDataTest]

    labels = [d['star_rating'] for d in reviewDataTest]
    
    # Autograder checks the MSE of your predictions and (some of) the simPrediction values

In [ ]:
#testQ6()

In [ ]:
def testQ7():
    alwaysPredictMean = [ratingMean for d in reviewDataTest]
    
    simPredictions = [homework2.predictRating(d['customer_id'],
                                              d['product_id'],
                                              ratingMean,
                                              reviewsPerUser,
                                              usersPerItem,
                                              itemsPerUser,
                                              userAverages,
                                              itemAverages) for d in reviewDataTest]
    

    q7Predictions = [homework2.predictRatingQ7(d['customer_id'],
                                               d['product_id'],
                                               ratingMean,
                                               reviewsPerUser,
                                               usersPerItem,
                                               itemsPerUser,
                                               userAverages,
                                               itemAverages) for d in reviewDataTest]
    
    labels = [d['star_rating'] for d in reviewDataTest]
    
    m1 = homework2.MSE(simPredictions, labels)
    m2 = homework2.MSE(q7Predictions, labels)
    m3 = homework2.MSE(alwaysPredictMean, labels)
    
    # Autograder checks whether your solution is better than either the Q6 or a naive solution
    return 1.0 * ((m2 < m1) and (m2 < m3))